In [84]:
import os
import pandas as pd
import openpyxl
from openpyxl.utils.datetime import from_excel
from datetime import datetime
import math
import re
import openpyxl
import os
import pandas as pd
import copy
from openpyxl.utils import get_column_letter
import numpy as np
import os
import glob
from pathlib import Path

import function       # モジュール全体をインポート
import importlib
importlib.reload(function)   # 変更を反映して再読み込み
from function import *       # 関数を名前空間に再登録

user_path = os.path.expanduser('~')

In [85]:
pd.set_option('display.max_columns', None)  # 全ての列を表示

In [86]:
# 入力してもらったマッピング表を読み込む
mapping_file_path = rf"{user_path}\OneDrive - トオカツフーズ株式会社\TKDX推進室\01_develop\【冷凍生産】_簡易ツール関連\（小杉）計画外移送のフォーマット作成\作成中\計画外移送登録_マッピング表.xlsx"
mapping_wb = openpyxl.load_workbook(mapping_file_path, data_only=True)
mapping_ws = mapping_wb["自動化用_マッピング"]
mapping_df = pd.DataFrame(mapping_ws.values)
mapping_df.columns = mapping_df.iloc[0]  # 1行目をヘッダーにセット
temp_file_path = rf"{user_path}\OneDrive - トオカツフーズ株式会社\TKDX推進室\01_develop\【冷凍生産】_簡易ツール関連\（小杉）計画外移送のフォーマット作成\作成中\計画外移送登録_フォーマット_ロット記入あり.xlsx"


c:\Users\kosugi-shiyu-t\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [87]:
mapping_df = pd.read_excel(mapping_file_path, sheet_name="自動化用_マッピング")
temp_df = pd.read_excel(temp_file_path)

# display(mapping_df.head())
# display(temp_df.head())

In [88]:
#mapping_dfから必要な項目を抽出し、辞書型で取得する()
hissu_mappings = mapping_df.columns.tolist()

#mapping_dfから行ごとにhissu_mappingsに含まれる項目を抽出し、辞書型で取得する
hissu_mapping_dicts = []
for _, row in mapping_df.iterrows():
    mapping_dict = {}
    for col in hissu_mappings:
        mapping_dict[col] = row[col]
    hissu_mapping_dicts.append(mapping_dict)
hissu_mapping_dicts

[{'書類種類': nan,
  '書類パターン': nan,
  '書類名': nan,
  'ヘッダー行': nan,
  '取消': '任意',
  '工場CD': '対象外',
  '移送No': '必須',
  '移送行': '必須',
  '移送区分CD': '必須',
  '移送区分': '対象外',
  '状態CD': '必須',
  '状態': '対象外',
  '移送日': '必須',
  '納入日': '必須',
  '在庫参照日': '必須',
  '元保管場所CD': '必須',
  '元保管場所': '対象外',
  '元保管棚CD': '必須',
  '元保管棚': '対象外',
  '品番': '必須',
  '品名': '対象外',
  '版': '必須',
  '規格': '対象外',
  '先工場CD': '対象外',
  '先工場': '対象外',
  '先保管場所CD': '必須',
  '先保管場所': '対象外',
  '先保管棚CD': '必須',
  '先保管棚': '対象外',
  '輸送便CD': '必須',
  '輸送便': '対象外',
  'ロットNo': '必須',
  'ロット枝番': '必須',
  '入数': '任意',
  '移送数': '必須',
  '単位区分CD': '必須',
  '単位': '対象外',
  '個数': '対象外',
  '個単位': '対象外',
  '換算数': '対象外',
  '備考': '任意',
  '丸め数（個）': '対象外',
  '最小手配（個）': '対象外',
  '担当者CD': '必須',
  '担当者': '対象外',
  '担当部門CD': '必須',
  '担当部門': '対象外',
  '出庫受払No': '必須※任意化予定',
  '渡し先名': nan,
  '賞味期限': nan,
  '入数.1': nan,
  '正箱数': nan,
  '個数.1': nan},
 {'書類種類': '配送依頼書',
  '書類パターン': '出庫兼配送依頼書(シンプル)',
  '書類名': '出庫兼配送依頼書_シンプル',
  'ヘッダー行': 31.0,
  '取消': nan,
  '工場CD': nan,
  '移送No': 0,

In [89]:
# 対象の拡張子リスト
extensions = ["xlsx", "xls", "csv"]

In [90]:
# マッピングから配送依頼書のデータを取得してくる

# 最終的に書類パターンごとのDataFrameを格納するリスト
merged_iraisho_list = []


# 書類パターンごとに処理
for mapping in hissu_mapping_dicts:

    #書類名がNaNの場合はスキップ
    if mapping["書類名"] is None or pd.isna(mapping["書類名"]):
        continue

    # 出庫報告書は後ほど処理する
    elif mapping['書類種類'] == "配送依頼書＋出庫報告書":
        continue

    else:
        print(f"{mapping['書類パターン']}のファイル処理を開始します。")

        #参照フォルダのパスの指定
        reference_folder_path = Path(rf"{user_path}\OneDrive - トオカツフーズ株式会社\TKDX推進室\01_develop\【冷凍生産】_簡易ツール関連\（小杉）計画外移送のフォーマット作成\作成中\計画外移送登録データ作成ツール\出庫兼配送依頼書および出庫報告書")

        # 書類名パターンに基づき配送依頼書を検索、読み込んで加工し、値の取得に用いるデータフレームを取得する
        results = load_target_iraisho_files(mapping, reference_folder_path, extensions)

        for target_df, target_wb, target_ws, create_temp_df in results:
            print(f"{len(target_df)} 行を読み込みました。")


            create_temp_df['書類名'] = mapping['書類名'] 
            create_temp_df["取消"] = mapping['取消']  #空白
            create_temp_df["工場CD"] = mapping['工場CD'] #空白
            create_temp_df["移送No"] = mapping["移送No"] #固定値
            create_temp_df['移送行'] = mapping['移送行'] #固定値
            create_temp_df['移送区分CD'] = mapping['移送区分CD'] #固定値
            create_temp_df['移送区分'] = mapping['移送区分'] #空白
            create_temp_df['状態CD'] = mapping['状態CD'] #固定値
            create_temp_df['状態'] = mapping['状態'] #空白

            create_temp_df['移送日(マッピング)'] = mapping['移送日'] #セル参照
            create_temp_df['移送日(形式統一前)'] = get_cell_value_by_cell_reference(target_ws, mapping['移送日']) #セル参照による取得後
            create_temp_df['移送日'] = process_date_value(create_temp_df['移送日(形式統一前)'], 1) #YYYY/MM/DDに統一後

            create_temp_df['納入日(マッピング)'] = mapping['納入日'] #セル参照
            create_temp_df['納入日(形式統一前)'] = get_cell_value_by_cell_reference(target_ws, mapping['納入日']) #セル参照による取得後
            create_temp_df['納入日'] = process_date_value(create_temp_df['納入日(形式統一前)'], 1) #YYYY/MM/DDに統一後
            create_temp_df['在庫参照日(マッピング)'] = mapping['在庫参照日'] #セル参照

            create_temp_df['在庫参照日(形式統一前)'] = get_cell_value_by_cell_reference(target_ws, mapping['在庫参照日']) #セル参照による取得後
            create_temp_df['在庫参照日'] = process_date_value(create_temp_df['在庫参照日(形式統一前)'], 1) #YYYY/MM/DDに統一後
            create_temp_df['元保管場所CD(マッピング)'] = mapping['元保管場所CD'] #セル参照

            create_temp_df['元保管場所CD'] = get_cell_value_by_cell_reference(target_ws, mapping['元保管場所CD']) #セル参照による取得後
            create_temp_df['元保管場所'] = mapping['元保管場所'] #空白
            create_temp_df['元保管棚CD'] = mapping['元保管棚CD'] #固定値
            create_temp_df['元保管棚'] = mapping['元保管棚'] #空白


            create_temp_df['品番(36追加前)'] = target_df[mapping['品番']] #カラム参照
            create_temp_df['品番(36追加前)_str'] = create_temp_df['品番(36追加前)'].astype(str) # 品番が入力されていない行を削除する作業。load_target_iraisho_files関数内で処理したはずだができなかったのでここで再度実行。
            create_temp_df = create_temp_df[create_temp_df['品番(36追加前)_str'].str.isdigit()]
            create_temp_df['品番'] = target_df[mapping['品番']].apply(lambda x: add_prefix(str(x))) #先頭に36を追加

            create_temp_df['品名'] = mapping['品名'] #空白
            create_temp_df['版'] = mapping['版'] #固定値
            create_temp_df['規格'] = mapping['規格'] #空白
            create_temp_df['先工場CD'] = mapping['先工場CD'] #空白
            create_temp_df['先工場'] = mapping['先工場'] #空白

            create_temp_df['先保管場所CD(マッピング)'] = mapping['先保管場所CD'] #セル参照

            create_temp_df['先保管場所'] = mapping['先保管場所'] #空白
            create_temp_df['先保管棚CD'] = mapping['先保管棚CD'] #固定値
            create_temp_df['先保管棚'] = mapping['先保管棚'] #空白
            create_temp_df['輸送便CD'] = mapping['輸送便CD'] #固定値
            create_temp_df['輸送便'] = mapping['輸送便'] #空白

            lot_no = target_df[mapping['ロットNo']] #カラム参照
            create_temp_df['ロットNo（型変換）'] = lot_no.apply(
                lambda x: str(int(float(x))) if str(x).replace('.', '', 1).isdigit() else str(x)
            )
            create_temp_df['ロットNo'] = create_temp_df['ロットNo（型変換）'].apply(lambda x: process_date_value(x, 2))

            
            create_temp_df['ロット枝番'] = mapping['ロット枝番'] #固定値
            create_temp_df['入数'] = mapping['入数'] #空白
            
            create_temp_df['移送数(マッピング)'] = mapping['移送数'] #計算式
            create_temp_df['移送数'] =calc_formula(target_df, mapping['移送数']) #計算式による取得後

            create_temp_df['単位区分CD'] = mapping['単位区分CD'] #固定値
            create_temp_df['単位'] = mapping['単位'] #空白
            create_temp_df['個数'] = mapping['個数'] #空白
            create_temp_df['個単位'] = mapping['個単位'] #空白
            create_temp_df['換算数'] = mapping['換算数'] #空白
            create_temp_df['備考'] = mapping['備考'] #空白
            create_temp_df['丸め数（個）'] = mapping['丸め数（個）'] #空白
            create_temp_df['最小手配（個）'] = mapping['最小手配（個）'] #空白
            create_temp_df['担当者CD'] = mapping['担当者CD'] #空白
            create_temp_df['担当者'] = mapping['担当者']  #空白
            create_temp_df['担当部門CD'] = mapping['担当部門CD'] #空白
            create_temp_df['担当部門'] = mapping['担当部門'] #固定値
            create_temp_df['出庫受払No'] = mapping['出庫受払No'] #空白
        

        


            # display(create_temp_df.head())
            
            merged_iraisho_list.append(create_temp_df)
        
        print(f"{mapping['書類パターン']}の処理が完了しました。")


# 依頼書データをすべてまとめたDataFrameを作成
merged_iraisho_df = pd.concat(merged_iraisho_list, ignore_index=True)

出庫兼配送依頼書(シンプル)のファイル処理を開始します。
検索結果: ['C:\\Users\\kosugi-shiyu-t\\OneDrive - トオカツフーズ株式会社\\TKDX推進室\\01_develop\\【冷凍生産】_簡易ツール関連\\（小杉）計画外移送のフォーマット作成\\作成中\\計画外移送登録データ作成ツール\\出庫兼配送依頼書および出庫報告書\\出庫兼配送依頼書_シンプル_北王フードベース（グローバルキッチン）.xlsx']
44 行を読み込みました。


C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_temp_df['品番'] = target_df[mapping['品番']].apply(lambda x: add_prefix(str(x))) #先頭に36を追加
C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_temp_df['品名'] = mapping['品名'] #空白
C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:66: SettingWithCopyWarning: 
A value is trying to be set o

型を確認します。0    2026-07-28 00:00:00
1    2026-07-18 00:00:00
2    2026-08-03 00:00:00
3    2026-07-23 00:00:00
4    2026-07-24 00:00:00
5    2025-11-18 00:00:00
6    2026-07-06 00:00:00
7    2026-08-04 00:00:00
8    2026-07-13 00:00:00
9    2026-08-21 00:00:00
Name: ロットNo（型変換）, dtype: object
型を確認します。ロットNo（型変換）
<class 'str'>    11
Name: count, dtype: int64
出庫兼配送依頼書(シンプル)の処理が完了しました。
出庫兼配送依頼書(ヨシケイ)のファイル処理を開始します。
検索結果: ['C:\\Users\\kosugi-shiyu-t\\OneDrive - トオカツフーズ株式会社\\TKDX推進室\\01_develop\\【冷凍生産】_簡易ツール関連\\（小杉）計画外移送のフォーマット作成\\作成中\\計画外移送登録データ作成ツール\\出庫兼配送依頼書および出庫報告書\\出庫兼配送依頼書_ヨシケイ確定手配書250822出荷分.xlsx']


c:\Users\kosugi-shiyu-t\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


33 行を読み込みました。
型を確認します。0       -
1    1724
2    1724
3       -
4    1717
5    1717
6    1626
7       -
8    1724
9    1725
Name: ロットNo（型変換）, dtype: object
型を確認します。ロットNo（型変換）
<class 'str'>    33
Name: count, dtype: int64
出庫兼配送依頼書(ヨシケイ)の処理が完了しました。
出庫兼配送依頼書(セットアップ)のファイル処理を開始します。
検索結果: ['C:\\Users\\kosugi-shiyu-t\\OneDrive - トオカツフーズ株式会社\\TKDX推進室\\01_develop\\【冷凍生産】_簡易ツール関連\\（小杉）計画外移送のフォーマット作成\\作成中\\計画外移送登録データ作成ツール\\出庫兼配送依頼書および出庫報告書\\出庫兼配送依頼書_セットアップ用.xlsx']
21 行を読み込みました。
型を確認します。0    20260226
1    20260227
2    20260228
3    20260301
4    20260302
5    20260303
6    20260304
7    20260305
8    20260306
9    20260307
Name: ロットNo（型変換）, dtype: object
型を確認します。ロットNo（型変換）
<class 'str'>    20
Name: count, dtype: int64
出庫兼配送依頼書(セットアップ)の処理が完了しました。
出庫報告_日の丸のファイル処理を開始します。
検索結果: ['C:\\Users\\kosugi-shiyu-t\\OneDrive - トオカツフーズ株式会社\\TKDX推進室\\01_develop\\【冷凍生産】_簡易ツール関連\\（小杉）計画外移送のフォーマット作成\\作成中\\計画外移送登録データ作成ツール\\出庫兼配送依頼書および出庫報告書\\ノムラフーズ様\u3000出庫報告書（日の丸）_20250904.xlsx']


C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_temp_df['品番'] = target_df[mapping['品番']].apply(lambda x: add_prefix(str(x))) #先頭に36を追加
C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_temp_df['品名'] = mapping['品名'] #空白
C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:66: SettingWithCopyWarning: 
A value is trying to be set o

5 行を読み込みました。
型を確認します。0    2026/07/18
1    2026/06/22
2    2026/07/06
3    2026/06/05
4    2026/06/10
Name: ロットNo（型変換）, dtype: object
型を確認します。ロットNo（型変換）
<class 'str'>    5
Name: count, dtype: int64
出庫報告_日の丸の処理が完了しました。
出庫兼配送依頼書_京都冷蔵(通販)のファイル処理を開始します。
検索結果: ['C:\\Users\\kosugi-shiyu-t\\OneDrive - トオカツフーズ株式会社\\TKDX推進室\\01_develop\\【冷凍生産】_簡易ツール関連\\（小杉）計画外移送のフォーマット作成\\作成中\\計画外移送登録データ作成ツール\\出庫兼配送依頼書および出庫報告書\\出庫兼配送依頼書_通販_湯葉丼の具\u3000出庫データ.xlsx']
44 行を読み込みました。
型を確認します。0    -
1    -
Name: ロットNo（型変換）, dtype: object
型を確認します。ロットNo（型変換）
<class 'str'>    2
Name: count, dtype: int64
出庫兼配送依頼書_京都冷蔵(通販)の処理が完了しました。
出庫兼配送依頼書_京都冷蔵(業務用)のファイル処理を開始します。
検索結果: ['C:\\Users\\kosugi-shiyu-t\\OneDrive - トオカツフーズ株式会社\\TKDX推進室\\01_develop\\【冷凍生産】_簡易ツール関連\\（小杉）計画外移送のフォーマット作成\\作成中\\計画外移送登録データ作成ツール\\出庫兼配送依頼書および出庫報告書\\出庫兼配送依頼書_業務_タイヘイ(北海道).xlsx', 'C:\\Users\\kosugi-shiyu-t\\OneDrive - トオカツフーズ株式会社\\TKDX推進室\\01_develop\\【冷凍生産】_簡易ツール関連\\（小杉）計画外移送のフォーマット作成\\作成中\\計画外移送登録データ作成ツール\\出庫兼配送依頼書および出庫報告書\\出庫兼配送依頼書_業務_タイヘイ(市川).xlsx', 'C:

C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_temp_df['品番'] = target_df[mapping['品番']].apply(lambda x: add_prefix(str(x))) #先頭に36を追加
C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_temp_df['品名'] = mapping['品名'] #空白
C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:66: SettingWithCopyWarning: 
A value is trying to be set o

44 行を読み込みました。
型を確認します。0    nan
1    nan
2    nan
3    nan
4    nan
5    nan
6    nan
Name: ロットNo（型変換）, dtype: object
型を確認します。ロットNo（型変換）
<class 'str'>    7
Name: count, dtype: int64
44 行を読み込みました。
型を確認します。0    nan
1    nan
2    nan
3    nan
4    nan
5    nan
Name: ロットNo（型変換）, dtype: object
型を確認します。ロットNo（型変換）
<class 'str'>    6
Name: count, dtype: int64
44 行を読み込みました。
型を確認します。0    nan
1    nan
2    nan
3    nan
4    nan
5    nan
6    nan
7    nan
8    nan
9    nan
Name: ロットNo（型変換）, dtype: object
型を確認します。ロットNo（型変換）
<class 'str'>    19
Name: count, dtype: int64
出庫兼配送依頼書_京都冷蔵(業務用)の処理が完了しました。
出庫報告_サンプル_第一倉庫冷蔵のファイル処理を開始します。
検索結果: ['C:\\Users\\kosugi-shiyu-t\\OneDrive - トオカツフーズ株式会社\\TKDX推進室\\01_develop\\【冷凍生産】_簡易ツール関連\\（小杉）計画外移送のフォーマット作成\\作成中\\計画外移送登録データ作成ツール\\出庫兼配送依頼書および出庫報告書\\【再】SHUKKA（第一倉庫冷蔵）.xls']
xls読込失敗(XLRDError)、CSVとして再試行します。
失敗: utf-8-sig -> UnicodeDecodeError
失敗: utf-8 -> UnicodeDecodeError
読み込み成功: cp932, 区切り='	', shape=(14, 29)
⚠ '商品コード'列が存在しません → C:\Users\kosugi-shiyu-t\OneDrive - トオ

C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_temp_df['品番'] = target_df[mapping['品番']].apply(lambda x: add_prefix(str(x))) #先頭に36を追加
C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  create_temp_df['品名'] = mapping['品名'] #空白
C:\Users\kosugi-shiyu-t\AppData\Local\Temp\ipykernel_13344\3304865913.py:66: SettingWithCopyWarning: 
A value is trying to be set o

In [91]:
# # 値取得後の配送依頼書データフレームのExcel出力
merged_iraisho_df.to_excel(rf"{user_path}\Desktop\計画外移送登録データ_配送依頼書の取得結果.xlsx", index=False)

In [92]:
# 出庫報告書を読みこみ、データフレームとして保持する

# 最終的に書類パターンごとのDataFrameを格納するリスト
merged_hokokusho_list = []

# 書類パターンごとに処理
for mapping in hissu_mapping_dicts:

    #書類名がNaNの場合はスキップ
    if mapping["書類名"] is None or pd.isna(mapping["書類名"]):
        continue

    # 出庫報告書（配送依頼書との突合に用いるもの）が処理対象
    elif mapping['書類種類'] != "配送依頼書＋出庫報告書":
        continue

    else:
        print(f"{mapping['書類パターン']}のファイル処理を開始します。")

        #参照フォルダのパスの指定
        reference_folder_path = Path(rf"{user_path}\OneDrive - トオカツフーズ株式会社\TKDX推進室\01_develop\【冷凍生産】_簡易ツール関連\（小杉）計画外移送のフォーマット作成\作成中\計画外移送登録データ作成ツール\出庫兼配送依頼書および出庫報告書")

        # 書類名パターンに基づき出庫報告書を検索、読み込んで、値の取得に用いるデータフレームを取得する
        results = load_target_hokokusho_files(mapping, reference_folder_path, extensions)

        for target_df, target_wb, target_ws, create_temp_df in results:
            # display(target_df.head())

            create_temp_df['書類名'] = mapping['書類名'] # 固定値

            # 日付形式をYYYY/MM/DDに統一（ゼロ埋め）
            create_temp_df['移送日'] = (
                pd.to_datetime(target_df[mapping['移送日']], errors='coerce')
                .dt.strftime('%Y/%m/%d')
            )
            
            create_temp_df['渡し先名'] = target_df[mapping['渡し先名']] # マッピング表には存在しないが報告書サマリには載せる項目、仮にマッピング済み

            create_temp_df['品番（マッピング）'] = target_df[mapping['品番']] # カラム参照
            create_temp_df['品番'] = create_temp_df['品番（マッピング）'].apply(lambda x: add_prefix(str(x))) # 先頭に36を追加

            # 第一倉庫冷蔵の出庫報告書には規格列がない
            if target_df.get('規格') is None:
                create_temp_df['規格'] = None
            else:
                create_temp_df['規格'] = target_df['規格'] # 移送数列のマッピングで指定される項目、マッピングしたい

            # 規格列から4桁ロット、入数、合を分離して取得
            count_series, total_series, lot_series = split_kikaku_series(target_df)
            create_temp_df['入数'] = count_series # 規格列から分離する項目
            create_temp_df['合'] = total_series # 規格列から分離する項目

            # 出庫個数・出庫端数列があるのは京都冷蔵のみ（標準化課題）
            for special_col in ['出庫個数', '出庫端数']:
                if target_df.get(special_col) is None:
                    create_temp_df[special_col] = None
                else:
                    create_temp_df[special_col] = target_df[special_col] # 移送数列のマッピングで指定される項目、マッピングしたい

            create_temp_df['移送数'] = calc_formula(target_df, mapping['移送数']) # 独自に計算する項目

            create_temp_df['賞味期限（マッピング）'] = target_df[mapping['賞味期限']] # マッピング表には存在しないが報告書サマリには載せる項目、仮にマッピング済み
            create_temp_df['賞味期限'] =  process_date_value(create_temp_df['賞味期限（マッピング）'], 1) #YYYY/MM/DDに統一後

            create_temp_df['4桁ロット'] = lot_series # 規格列から分離する項目

            create_temp_df['ロットNo'] = create_temp_df.apply(
                lambda create_temp_df: make_lot_no(create_temp_df['4桁ロット'], create_temp_df['賞味期限']), axis=1
            ) # 独自に計算する項目

            # display(create_temp_df.head())
            
            merged_hokokusho_list.append(create_temp_df)
        
        print(f"{mapping['書類パターン']}の処理が完了しました。")


# 依頼書データをすべてまとめたDataFrameを作成
merged_hokokusho_df = pd.concat(merged_hokokusho_list, ignore_index=True)
# display(merged_hokokusho_df)

出庫報告_サンプル_京都冷蔵(通販)のファイル処理を開始します。
検索結果: ['C:\\Users\\kosugi-shiyu-t\\OneDrive - トオカツフーズ株式会社\\TKDX推進室\\01_develop\\【冷凍生産】_簡易ツール関連\\（小杉）計画外移送のフォーマット作成\\作成中\\計画外移送登録データ作成ツール\\出庫兼配送依頼書および出庫報告書\\通販出庫報告書_（京都冷蔵）_出庫報告書_3475_20250821081346813.csv']
失敗: utf-8-sig -> UnicodeDecodeError
失敗: utf-8 -> UnicodeDecodeError
読み込み成功: cp932, 区切り=',', shape=(10, 28)
10 行を読み込みました: 通販出庫報告書_（京都冷蔵）_出庫報告書_3475_20250821081346813.csv
出庫報告_サンプル_京都冷蔵(通販)の処理が完了しました。
出庫報告_サンプル_京都冷蔵(業務用)のファイル処理を開始します。
検索結果: ['C:\\Users\\kosugi-shiyu-t\\OneDrive - トオカツフーズ株式会社\\TKDX推進室\\01_develop\\【冷凍生産】_簡易ツール関連\\（小杉）計画外移送のフォーマット作成\\作成中\\計画外移送登録データ作成ツール\\出庫兼配送依頼書および出庫報告書\\業務用出庫報告書_（京都冷蔵）_3474_20250917081210810.csv']
失敗: utf-8-sig -> UnicodeDecodeError
失敗: utf-8 -> UnicodeDecodeError
読み込み成功: cp932, 区切り=',', shape=(142, 28)
142 行を読み込みました: 業務用出庫報告書_（京都冷蔵）_3474_20250917081210810.csv
出庫報告_サンプル_京都冷蔵(業務用)の処理が完了しました。
出庫報告_サンプル_京都冷蔵(ヨシケイ)のファイル処理を開始します。
検索結果: ['C:\\Users\\kosugi-shiyu-t\\OneDrive - トオカツフーズ株式会社\\TKDX推進室\\01_develop\\【冷凍生産】_簡易ツール関連\

In [93]:
# 値取得後の配送依頼書データフレームのExcel出力
merged_hokokusho_df.to_excel(rf"{user_path}\Desktop\計画外移送登録データ_出庫報告書サマリ.xlsx", index=False)

In [94]:
# ① merged_iraisho_dfのうちロットNoの有無でフィルタリングし、記載のあるもの（iraisho_df_with_lot_no）とないもの（iraisho_df_without_lot_no）を分ける

# ロットNoが数字のみで構成されているかどうかでフィルタリング
merged_iraisho_df['ロットNo_str'] = merged_iraisho_df['ロットNo'].astype(str)
lot_lacking_iraisho_df = merged_iraisho_df[~merged_iraisho_df['ロットNo_str'].str.isdigit()].copy()
lot_designated_iraisho_df = merged_iraisho_df[merged_iraisho_df['ロットNo_str'].str.isdigit()].copy()
lot_lacking_iraisho_df.drop(columns=['ロットNo_str'], inplace=True)
lot_designated_iraisho_df.drop(columns=['ロットNo_str'], inplace=True)

In [95]:
# ②（iraisho_df_with_lot_no）=タイプA・Cの場合、不要カラムを削除し、そのままアウトプット出力
required_columns = []
for col in lot_designated_iraisho_df.columns:
    if col in hissu_mapping_dicts[0] and hissu_mapping_dicts[0][col] == "必須":
        required_columns.append(col)

iraisho_df_with_lot_no = lot_designated_iraisho_df[required_columns].copy()
iraisho_df_with_lot_no.to_excel(rf"{user_path}\Desktop\計画外移送登録データ_ロット記入あり.xlsx", index=False)

In [96]:
# ③（lot_lacking_iraisho_df）＝タイプBの場合、merged_hokokusho_dfに対し、移送日＋品番でフィルターした結果に応じて、以下の処理をおこなう。
# （１）1種類のロットならそのロットを補填	→（iraisho_df_with_lot_no）に追加
# （２）2種類以上のロットがあったら、,(カンマ)区切りでロットを列挙し、ユーザーはそれを見てどのロットを補填するか判断


single_lot_filled_list = []
several_lot_filled_list = []
lot_unfilled_list = []

for _, row in lot_lacking_iraisho_df.iterrows():
    temp_iraisho_date = row["移送日"]
    temp_iraisho_item = row["品番"]

    # 突合
    matched_hokokusho_rows = merged_hokokusho_df[
        (merged_hokokusho_df["移送日"] == temp_iraisho_date) &
        (merged_hokokusho_df["品番"] == temp_iraisho_item)
    ]

    unique_lot_nos = matched_hokokusho_rows['ロットNo'].unique()

    # --- ① 1種類のロットが見つかった場合 ---
    if len(unique_lot_nos) == 1:
        lot_to_fill = unique_lot_nos[0]
        lot_lacking_iraisho_df.at[row.name, "ロットNo"] = lot_to_fill

        # この行のみ抽出してリストに追加
        single_lot_filled_list.append(lot_lacking_iraisho_df.loc[[row.name], required_columns])

        print(f"１種類のロットが見つかりました: {lot_to_fill}")

    # --- ② 複数ロットが見つかった場合 ---
    elif len(unique_lot_nos) > 1:
        lot_list_str = ', '.join(map(str, unique_lot_nos))
        lot_lacking_iraisho_df.at[row.name, "ロットNo"] = lot_list_str

        # この行のみ抽出してリストに追加
        several_lot_filled_list.append(lot_lacking_iraisho_df.loc[[row.name], required_columns])

        print(f"複数種類のロットが見つかりました: {lot_list_str}")

    else:
        lot_unfilled_list.append(lot_lacking_iraisho_df.loc[[row.name], required_columns])
        print(f"ロットNoの検出に失敗しました: {unique_lot_nos}")

# --- 出力処理（各リストを結合してからExcel保存） ---
if single_lot_filled_list:
    single_lot_filled_df = pd.concat(single_lot_filled_list, ignore_index=True)
    single_lot_filled_df.to_excel(
        rf"{user_path}\Desktop\計画外移送登録データ_ロット記入なし（補填成功）.xlsx",
        index=False
    )

if several_lot_filled_list:
    several_lot_filled_df = pd.concat(several_lot_filled_list, ignore_index=True)
    several_lot_filled_df.to_excel(
        rf"{user_path}\Desktop\計画外移送登録データ_ロット記入なし（候補出力）.xlsx",
        index=False
    )


if lot_unfilled_list:
    lot_unfilled_df = pd.concat(lot_unfilled_list, ignore_index=True)
    lot_unfilled_df.to_excel(
        rf"{user_path}\Desktop\計画外移送登録データ_ロット記入なし（補填失敗）.xlsx",
        index=False
    )

１種類のロットが見つかりました: 20260831-1724
複数種類のロットが見つかりました: 20260831-1716, 20260831-1717, 20260831-1718
複数種類のロットが見つかりました: 20260831-1724, 20260831-1725
１種類のロットが見つかりました: 20260731-1624
１種類のロットが見つかりました: 20260831-1724
複数種類のロットが見つかりました: 20260831-1716, 20260831-1717, 20260831-1718
１種類のロットが見つかりました: 20260731-1626
１種類のロットが見つかりました: 20260831-1724
複数種類のロットが見つかりました: 20260831-1716, 20260831-1717, 20260831-1718
複数種類のロットが見つかりました: 20260831-1724, 20260831-1725
１種類のロットが見つかりました: 20260731-1624
複数種類のロットが見つかりました: 20260417, 20260511
複数種類のロットが見つかりました: 20260417, 20260511
１種類のロットが見つかりました: 20260808
１種類のロットが見つかりました: 20260722
１種類のロットが見つかりました: 20260901
１種類のロットが見つかりました: 20260818
１種類のロットが見つかりました: 20260729
複数種類のロットが見つかりました: 20260731, 20260808
１種類のロットが見つかりました: 20260821
１種類のロットが見つかりました: 20260806
１種類のロットが見つかりました: 20260819
複数種類のロットが見つかりました: 20260818, 20260821
１種類のロットが見つかりました: 20260808
複数種類のロットが見つかりました: 20260803, 20260818
複数種類のロットが見つかりました: 20260807, 20260818
１種類のロットが見つかりました: 20260724
１種類のロットが見つかりました: 20260821
１種類のロットが見つかりました: 20260710
